In [2]:
from kan import * 

In [3]:
from tensorflow.keras.datasets import mnist
import numpy as np

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Flatten the images 
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Filter training and test sets for digits 0 and 1
train_mask = (y_train == 0) | (y_train == 1)
test_mask = (y_test == 0) | (y_test == 1)                                     

X_train_filtered = X_train[train_mask]
y_train_filtered = y_train[train_mask]

X_test_filtered = X_test[test_mask]
y_test_filtered = y_test[test_mask]


In [4]:
X_train.shape

(60000, 784)

In [5]:
(Xxx_train, yyy_train), (Xxx_test, yyy_test) = mnist.load_data()

In [6]:
Xxx_train.shape

(60000, 28, 28)

In [9]:
X_train_filtered.shape

(12665, 784)

In [10]:
y_train_filtered.shape

(12665,)

In [8]:
y_train.shape

(60000,)

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

dataset = {}
train_input, train_label = X_train_filtered , y_train_filtered
test_input, test_label = X_test_filtered , y_test_filtered 

dtype = torch.get_default_dtype()
dataset['train_input'] = torch.from_numpy(train_input).type(dtype).to(device)
dataset['test_input'] = torch.from_numpy(test_input).type(dtype).to(device)     
dataset['train_label'] = torch.from_numpy(train_label[:,None]).type(dtype).to(device)
dataset['test_label'] = torch.from_numpy(test_label[:,None]).type(dtype).to(device)

X = dataset['train_input']   
y = dataset['train_label']


cpu


In [13]:
model = KAN(width=[784,10,1], grid=10, k=5, device=device)

def train_acc():
    return torch.mean((torch.round(model(dataset['train_input'])[:,0]) == dataset['train_label'][:,0]).type(dtype))

def test_acc():
    return torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).type(dtype))

results = model.fit(dataset, opt="LBFGS", steps=20, metrics=(train_acc, test_acc));     
results['train_acc'][-1], results['test_acc'][-1]     

checkpoint directory created: ./model
saving model version 0.0


| train_loss: 5.83e-02 | test_loss: 5.06e-02 | reg: 1.31e+02 | : 100%|█| 20/20 [1:32:50<00:00, 278.5


saving model version 0.1


(0.9984998106956482, 0.9990543723106384)

Automatic Symbolic Regression

## Classification formulation

Let's then treat the problem as a classification problem (output dimension = 2, CrossEntropy loss). 